# CI/CD

## Github Action Workflow

GitHub Actions workflow允许在GitHub仓库中自动化任务。 

workflow文件采用Yaml格式,放在代码仓库的.github/workflows目录下。

### workflow Yaml文件书写

基本结构包括:

- name: workflow名称
- on: 触发事件,如push、pull request
- jobs: 要执行的任务
  - uses: 使用操作 
  - run: 运行命令和脚本
  - env: 设置环境变量

### Python workflow示例

**测试和发布PyPI包**

```yaml
on: push
jobs:
  build:
    runs-on: ubuntu-latest
    steps:
    - uses: actions/checkout@v2
    - uses: actions/setup-python@v2
      with: 
        python-version: 3.8
    - run: pip install poetry
    - run: poetry install
    - run: poetry run pytest
    - uses: pypa/gh-action-pypi-publish@v1.4
      with:
        password: ${{ secrets.PYPI_API_TOKEN }}
```

在push时触发,安装依赖,运行测试,并发布到PyPI。

**构建和部署Docker镜像**

```yaml
on: push
jobs:
  build:
    runs-on: ubuntu-latest
    steps:
    - uses: actions/checkout@v2
    - uses: docker/build-push-action@v1
      with:
        username: ${{secrets.DOCKER_USERNAME}}
        password: ${{secrets.DOCKER_PASSWORD}}
        repository: myhub/myimage
        tag_with_ref: true
```

在push时构建Docker镜像,并push到Docker Hub。


在 GitHub Actions 的 workflow 文件中,可以通过 secrets 机制来安全地传入诸如Docker Hub用户名密码等敏感信息。

具体做法是:

1. 在仓库的 Settings - Secrets 页面中添加一个 secret,例如名为 DOCKER_USERNAME,值为你的 Docker Hub 用户名。

2. 在 workflow 文件中,通过 `${{secrets.DOCKER_USERNAME}}` 来引用该 secret。

例如:

```yaml
steps:
- uses: docker/build-push-action@v1
  with:
    username: ${{secrets.DOCKER_USERNAME}} 
    password: ${{secrets.DOCKER_PASSWORD}}
```

这样 GitHub Actions 就可以在运行时自动替换该变量为你设置的真实用户名,而不会直接暴露在文件中。

secrets 可以被设置为仓库、组织或企业级变量,访问权限可以灵活配置。

除了 Docker Hub,也可以用它来传入 PyPI、npm 等服务的令牌、秘钥等信息。

这样可以确保这些敏感数据的安全,避免直接写入到仓库文件中。

### Demo for Action workflow

```yml
name: Python CI

on: 
  push:
    branches: [main]
  pull_request:
    branches: [main]

jobs:
  build:

    runs-on: ubuntu-latest

    steps:
    - uses: actions/checkout@v2
    - name: Set up Python 3.8
      uses: actions/setup-python@v2
      with:
        python-version: 3.8
    - name: Install dependencies
      run: |
        python -m pip install --upgrade pip
        pip install -r requirements.txt
    - name: Run pre-commit
      run: |
        pre-commit install
        pre-commit run --all-files
    - name: Test with pytest
      run: |
        pip install pytest
        pytest

解释说明：

1. 在 GitHub Actions 的 workflow 文件中,uses: actions/checkout@v2 这行的作用是检出(checkout)代码仓库。
2. run后使用 "｜" 表示换行输入命令。